In [ ]:

import os
import cv2
import numpy as np
import sys

sys.path.append('../')

import torchlm
# from torchlm.tools import faceboxesv2
# from torchlm.models import pipnet

import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from models.LawDNet_new import LawDNet 
from collections import OrderedDict
import warnings 
# from cfg import *
# from tools import *

from torchlm.tools import faceboxesv2
from torchlm.models import pipnet

# from torch_affine_ops import *
from tensor_processing import *
warnings.filterwarnings("ignore")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def read_video_np(video_path):
    cap = cv2.VideoCapture(video_path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    H, W = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print('video length: ', length)
    print('video size: ','H--', H, 'W--',W)
    

    frames = []
    frames_id = 1
    while True:
        ret, frame = cap.read()
        if ret and frames_id <= 1500:
            frames_id += 1 
            frames.append(frame)
        else:
            break
    cap.release()
    return frames


In [ ]:
video_path = '../template/1-坐.mp4'
video_frames = read_video_np(video_path)
video_frames = np.array(video_frames, dtype=np.float32)
video_frames = video_frames[:, :, :, ::-1] # BGR to RGB

In [ ]:
# audio
deepspeech_tensor = torch.load('../template/longaudio_deepspeech_tensor_all.pt')
B = 10
len_out = (np.min([len(video_frames), deepspeech_tensor.shape[0]])//B)*B
video_frames = video_frames[:len_out]
deepspeech_tensor = deepspeech_tensor[:len_out]

In [ ]:
# load model
net_g = LawDNet(3,15,29).to(device)
checkpoint = torch.load('/home/hermit_delta/data/hermit_delta/DINet-update/checkpoints/实验11-smoothsqmask-netG_model_epoch_25.pth') 
state_dict = checkpoint['state_dict']['net_g']
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:]  # remove module.
    new_state_dict[name] = v
net_g.load_state_dict(new_state_dict)
net_g.eval()
facealigner = FaceAlign(ratio=1.6, device=device)
sqmasker = SmoothSqMask()
#sqmasker = SmoothSqMask(radius=30,sigma=10,standard_shape=(416,320),padding='same')

In [ ]:
#### landmark
torchlm.runtime.bind(faceboxesv2(device=device))  # set device="cuda" if you want to run with CUDA
# set map_location="cuda" if you want to run with CUDA
torchlm.runtime.bind(
  pipnet(backbone="resnet18", pretrained=True,  
         num_nb=10, num_lms=68, net_stride=32, input_size=256,
         meanface_type="300w", map_location=device.__str__(), checkpoint=None) 
) # will auto download pretrained weights from latest release if pretrained=True
landmarks,_ = torchlm.runtime.forward(video_frames[0])


landmarks_list = []
for i in range(len(video_frames)):
    landmarks, bboxes = torchlm.runtime.forward(video_frames[i])
    trust = bboxes[:,4]
    faithful_index = np.argsort(trust)[-1]
    landmarks_list.append(landmarks[faithful_index,:,:])


landmarks_list = np.array(landmarks_list)

In [ ]:
# uniform reference from random 5 frames

reference_index = torch.randint(0, len(video_frames), (5,)).tolist()

reference_tensor = torch.from_numpy(video_frames[reference_index]).to(device)
reference_tensor = reference_tensor.permute(0,3,1,2)
reference_landmarks = torch.from_numpy(landmarks_list[reference_index]).to(device)

reference_tensor,_,_ = facealigner(reference_tensor, reference_landmarks,out_W=320)

plt.imshow(reference_tensor[0].permute(1,2,0).cpu().numpy().astype(np.uint8))


In [ ]:
def reference(model,masked_source,reference,audio_tensor):
    with torch.no_grad():
        output = model(masked_source,reference,audio_tensor)
        return output

In [ ]:
outframes = np.zeros_like(video_frames)

for i in range(len_out//B):
    source_tensor = torch.from_numpy(video_frames[i*B:(i+1)*B].copy()).to(device)
    source_tensor = source_tensor.permute(0,3,1,2)
    landmarks_tensor = torch.from_numpy(landmarks_list[i*B:(i+1)*B]).to(device)
    feed_tensor,_,affine_matrix = facealigner(source_tensor, landmarks_tensor,out_W=320)
    _,C,H,W = feed_tensor.shape

    feed_tensor_masked = sqmasker(feed_tensor)

    reference_tensor_B = feed_tensor[2:7].view(1,5*3,H,W).repeat(B,1,1,1)

    #reference_tensor_B = reference_tensor.unsqueeze(0).repeat(B,1,1,1,1).view(B,5*3,H,W)

    audio_tensor = deepspeech_tensor[i*B:(i+1)*B].to(device)

    output_B = reference(net_g,feed_tensor_masked/255.0,reference_tensor_B/255.0,audio_tensor)
    # covert half into float
    output_B = output_B.float().clamp_(0,1)*255.0
    
    # feathering
    output_B = facealigner.feathering(output_B,feed_tensor)
    # recover
    outframes_B = facealigner.recover(output_B,source_tensor,affine_matrix)

    outframes_B = outframes_B.permute(0,2,3,1).cpu().numpy().astype(np.uint8)

    outframes[i*B:(i+1)*B] = outframes_B

    
    

In [ ]:
outframes = outframes.astype(np.uint8) 
outframes = outframes[:,:,:,::-1] # RGB to BGR

In [ ]:
# export video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter('output.mp4', fourcc, 25, (video_frames.shape[2],video_frames.shape[1]),True)
for i in range(len_out):
    videoWriter.write(outframes[i])
videoWriter.release()



In [ ]:
plt.imshow(outframes[i])

In [ ]:
plt.imshow(feed_tensor[0].permute(1,2,0).cpu().numpy().astype(np.uint8))

In [ ]:
import os
import uuid
from ffmpy import FFmpeg

In [ ]:

# 视频添加音频
def video_add_audio(video_path: str, audio_path: str, output_dir: str):
    _ext_video = os.path.basename(video_path).strip().split('.')[-1]
    _ext_audio = os.path.basename(audio_path).strip().split('.')[-1]
    if _ext_audio not in ['mp3', 'wav']:
        raise Exception('audio format not support')
    _codec = 'copy'
    if _ext_audio == 'wav':
        _codec = 'aac'
    result = os.path.join(
        output_dir, '{}.{}'.format(
            uuid.uuid4(), _ext_video))
    ff = FFmpeg(
        inputs={video_path: None, audio_path: None},
        outputs={result: '-map 0:v -map 1:a -c:v copy -c:a {} -shortest'.format(_codec)})
    print(ff.cmd)
    ff.run()
    return result

In [ ]:
video_add_audio('output.mp4', '../template/RD_Radio14_000_corrected.wav', './')